In [88]:
from os import listdir
from os.path import isfile, join

statdir = "playerstats"
files = [join(statdir, f) for f in listdir(statdir) if isfile(join(statdir, f))]

In [89]:
import json

playerdata = []
for jf in files:
    with open(jf) as f:
        playerdata.append(json.load(f))

In [90]:
flatdata = [item for sublist in playerdata for item in sublist]

In [91]:
import numpy as np

ignorethese = ('MatchID', 'Lane', 'Role')

def filterdata(data, lane, role):
    return filter(lambda stat: (lane == "ANY" or stat["Lane"] == lane) and 
                                    (role == "ANY" or stat["Role"] == role), 
                                   data)

def createAvgStats(stats, lane, role):
    avgstats = {}
    filteredstats = filterdata(stats, lane, role)
    if len(filteredstats) == 0:
        return None
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in avgstats:
                    avgstats[key] += stat[key]
                else:
                    avgstats[key] = stat[key]
    for key in avgstats:
        avgstats[key] /= float(len(filteredstats))
    avgstats["Lane"] = lane
    avgstats["Role"] = role
    return avgstats

def createPlayersAvgStats(playerdata, lane, role):
    arr = []
    for p in playerdata:
        ap = createAvgStats(p, lane, role)
        if ap:
            ap["GamesPlayed"] = len(filterdata(p, lane, role))
            arr.append(ap)
    return createAvgStats(arr, lane, role)

def createStatsDistribution(stats, lane, role):
    statdistribution = {}
    filteredstats = filterdata(stats, lane, role)
    if len(filteredstats) == 0:
        return None
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in statdistribution:
                    statdistribution[key].append(stat[key])
                else:
                    statdistribution[key] = [stat[key]]
        
    statdistribution["Lane"] = lane
    statdistribution["Role"] = role
    return statdistribution

def createStdStats(stats, lane, role):
    sd = createStatsDistribution(stats, lane, role)
    
    if sd == None:
        return None
    
    stds = {}
    for key in sd:
        if key == "Lane" or key == "Role":
            stds[key] = sd[key]
        else:
            stds[key] = np.std(sd[key])
    
    return stds

def createPlayersStdStats(playerdata, lane, role):
    arr = []
    for p in playerdata:
        ap = createAvgStats(p, lane, role)
        if ap:
            ap["GamesPlayed"] = len(filterdata(p, lane, role))
            arr.append(ap)
    return createStdStats(arr, lane, role)

In [92]:
useflat = False
avgstatdata = []
if(not useflat):
    avgstatdata.append(createPlayersAvgStats(playerdata, "TOP", "SOLO"))
    avgstatdata.append(createPlayersAvgStats(playerdata, "JUNGLE", "NONE"))
    avgstatdata.append(createPlayersAvgStats(playerdata, "MIDDLE", "SOLO"))
    avgstatdata.append(createPlayersAvgStats(playerdata, "BOTTOM", "DUO_CARRY"))
    avgstatdata.append(createPlayersAvgStats(playerdata, "BOTTOM", "DUO_SUPPORT"))
    avgstatdata.append(createPlayersAvgStats(playerdata, "ANY", "ANY"))
else:
    avgstatdata.append(createAvgStats(flatdata, "TOP", "SOLO"))
    avgstatdata.append(createAvgStats(flatdata, "JUNGLE", "NONE"))
    avgstatdata.append(createAvgStats(flatdata, "MIDDLE", "SOLO"))
    avgstatdata.append(createAvgStats(flatdata, "BOTTOM", "DUO_CARRY"))
    avgstatdata.append(createAvgStats(flatdata, "BOTTOM", "DUO_SUPPORT"))
    avgstatdata.append(createAvgStats(flatdata, "ANY", "ANY"))

In [93]:
with open('player_stats_mean.json', 'w') as f:
    json.dump(avgstatdata, f)

In [94]:
stddata = []
if(not useflat):
    stddata.append(createPlayersStdStats(playerdata, "TOP", "SOLO"))
    stddata.append(createPlayersStdStats(playerdata, "JUNGLE", "NONE"))
    stddata.append(createPlayersStdStats(playerdata, "MIDDLE", "SOLO"))
    stddata.append(createPlayersStdStats(playerdata, "BOTTOM", "DUO_CARRY"))
    stddata.append(createPlayersStdStats(playerdata, "BOTTOM", "DUO_SUPPORT"))
    stddata.append(createPlayersStdStats(playerdata, "ANY", "ANY"))
else:
    stddata.append(createStdStats(flatdata, "TOP", "SOLO"))
    stddata.append(createStdStats(flatdata, "JUNGLE", "NONE"))
    stddata.append(createStdStats(flatdata, "MIDDLE", "SOLO"))
    stddata.append(createStdStats(flatdata, "BOTTOM", "DUO_CARRY"))
    stddata.append(createStdStats(flatdata, "BOTTOM", "DUO_SUPPORT"))
    stddata.append(createStdStats(flatdata, "ANY", "ANY"))

In [95]:
with open('player_stats_std.json', 'w') as f:
    json.dump(stddata, f)

In [18]:
allgames = [item for sublist in playerdata for item in sublist]

In [22]:
print len(filterdata(allgames, "TOP", "SOLO"))
print len(filterdata(allgames, "TOP", "ANY"))
print len(filterdata(allgames, "MIDDLE", "SOLO"))
print len(filterdata(allgames, "MIDDLE", "ANY"))

24890
24973
22636
23248


In [96]:
len(allgames)

NameError: name 'allgames' is not defined

In [100]:
arr = []
for p in playerdata:
    ap = createAvgStats(p, "ANY", "ANY")
    if ap:
        ap["GamesPlayed"] = len(p)
        arr.append(ap)

In [103]:
arr2 = [x["Winner"] for x in arr]

In [109]:
arr2[1]

0.5314285714285715